In [1]:
import numpy as np
import scipy
import math

In [2]:
def random_params():
    return {
        'a': np.random.uniform(0, 1),
        'b': np.random.uniform(0, 1),
        'r': np.random.uniform(0, 1),
    }

In [3]:
def f(x, params):
    return  x * np.log( 1 + params['a']/ x ) + 1 / params['r'] * np.log( 1 + params['b'] * x )

In [4]:
params1 = random_params()
params2 = random_params()

print(params1)
print(params2)

{'a': 0.9494742324362261, 'b': 0.6685633077377735, 'r': 0.7227403455317668}
{'a': 0.1367742459011866, 'b': 0.8957397001086203, 'r': 0.26823546232406603}


In [5]:
print(params1)
print(params2)

{'a': 0.9494742324362261, 'b': 0.6685633077377735, 'r': 0.7227403455317668}
{'a': 0.1367742459011866, 'b': 0.8957397001086203, 'r': 0.26823546232406603}


In [6]:
# find optimal x(x1, x2) to minimize f(x1) + f(x2) subject to x1 + x2 <= 1

def sum(x):
    return f(x[0], params1) + f(x[1], params2)


def constraint(x):
    return 2 - x[0] - x[1]


starting_point = [0.1, 0.2]

cons = [{'type': 'ineq', 'fun': constraint},
        {'type': 'ineq', 'fun': lambda x: x[0]},
        {'type': 'ineq', 'fun': lambda x: x[1]},
        # ({'type': 'ineq', 'fun': lambda x: 1 - 2 * x[0] - 1* x[1]}),
        # ({'type': 'ineq', 'fun': lambda x: 1 - x[1]}) 
        ]

res = scipy.optimize.minimize(
    sum, starting_point, constraints=cons, bounds=[(0.1, 1), (0.1, 1)], method='SLSQP')

In [7]:
res

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.7306429886140535
       x: [ 1.000e-01  1.000e-01]
     nit: 2
     jac: [ 2.313e+00  3.349e+00]
    nfev: 6
    njev: 2

In [8]:
sum([2., 2.])

5.910976887947747

In [9]:
a = np.array([1, 2, 3])

A = np.array([[1, 2, 3], [4, 5, 6]])
B = np.array([10, 20, 30])

In [10]:
def main(unit_func, A, b, initial_x):
    def objective_func(x):
        sum = 0
        for xi in x:
            params = random_params()
            unit = f(xi, params=params)
            sum += unit
        return sum

    def get_constraint():
        return {'type': 'ineq', 'fun': lambda x: np.sum(x) - 1}


In [11]:
from scipy.optimize import linprog
from scipy.optimize import minimize
from scipy.optimize import Bounds
from scipy.optimize import LinearConstraint
from scipy.optimize import shgo

# Maximize F = sum of f_i(x_i) for i = 1 to N
# f_i(x_i) = x_i * log2(1 + a_i/x_i) + 1/r_i * log2(1 + b_i * x_i)
# Constraints:
#   sum of x_i <= B
#   x_i >= 0 for all i
class Solution:
    def __init__(self, N, B, method = 'trust-constr'):
        self.N = N
        self.B = B
        data = open('./data/input_{}_{}.txt'.format(N, B), 'r').read().splitlines()[1:]
        self.a = np.array(list(map(float, data[0].split())))
        self.b = np.array(list(map(float, data[1].split())))
        self.r = np.array(list(map(float, data[2].split())))

        self.objective = lambda x: self.F(x)
        self.objective_der = lambda x: self.F_der(x)
        self.objective_hess = lambda x: self.F_hess(x)
        self.result = None
        self.method = method

    def F(self, x):
        return -np.sum(x * np.log2(1 + self.a / x) + 1 / self.r * np.log2(1 + self.b * x))
    
    def F_der(self, x):
        return -np.log2(1 + self.a/x) + self.a / ((x+self.a)*np.log(2)) + self.b / (self.r * np.log(2) * (1 + self.b * x))
    
    def F_hess(self, x):
        hess = np.zeros((self.N, self.N))
        for i in range(self.N):
            hess[i, i] = (self.a[i]**2 + self.a[i]*x[i]*(1-np.log(2)))/(x[i]*(x[i]+self.a[i])**2*np.log(2)) + self.b[i]**2 / (self.r[i] * np.log(2) * (1 + self.b[i] * x[i])**2)
        return hess

    def solve(self, random_start = False):
        cons = LinearConstraint(np.ones(self.N), 0, self.B)
        bounds = Bounds(np.zeros(self.N), np.inf * np.ones(self.N))
        if random_start:
            x0 = []
            for i in range(self.N):
                x0.append(np.random.uniform(0, self.B - np.sum(x0)))
            x0 = np.array(x0)
        else:
            x0 = np.ones(self.N)
        self.result = minimize(self.objective, x0, method=self.method, jac=self.objective_der, hess=self.objective_hess, constraints=cons, bounds=bounds)
        return self.result
    
   
    

    
    def print_result(self):
        #print('N = {}, B = {}'.format(self.N, self.B))
        #print('a = {}'.format(self.a))
        #print('b = {}'.format(self.b))
        #print('r = {}'.format(self.r))
        print('x = {}'.format(self.result.x))
        print('sum of x = {:.4f}'.format(np.sum(self.result.x)))
        print('Max F = {:.4f}'.format(-self.result.fun))

In [12]:
sol = Solution(5, 200)

In [13]:
sol.solve()

           message: `xtol` termination condition is satisfied.
           success: True
            status: 2
               fun: -25.146425296326463
                 x: [ 4.196e+00  7.710e+00  5.937e-03  1.842e+02  1.270e+00]
               nit: 122
              nfev: 197
              njev: 35
              nhev: 35
          cg_niter: 242
      cg_stop_cond: 2
              grad: [ 9.719e-05  7.655e-05  1.479e-01 -8.971e-04  3.387e-03]
   lagrangian_grad: [-3.204e-05 -5.360e-05  5.207e-06 -1.028e-03  2.010e-03]
            constr: [array([ 1.974e+02]), array([ 4.196e+00,  7.710e+00,  5.937e-03,  1.842e+02,
                            1.270e+00])]
               jac: [array([[ 1.000e+00,  1.000e+00,  1.000e+00,
                             1.000e+00,  1.000e+00]]), array([[ 1.000e+00,  0.000e+00, ...,  0.000e+00,
                             0.000e+00],
                           [ 0.000e+00,  1.000e+00, ...,  0.000e+00,
                             0.000e+00],
                     

In [14]:
sol.print_result()

x = [4.19599687e+00 7.70963725e+00 5.93707800e-03 1.84194838e+02
 1.27023434e+00]
sum of x = 197.3766
Max F = 25.1464


# Trong Linear Progamming, n an m rang buoc thi so dinh toi da can duyet la C(m,n) 